In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="Speed Op Google Signin-ff8d3702ea7d.json"

In [2]:
from google.cloud import bigquery


In [3]:
client = bigquery.Client()

In [4]:
hn_dataset_ref = client.dataset('hacker_news', project='bigquery-public-data')

In [5]:
type(hn_dataset_ref)

google.cloud.bigquery.dataset.DatasetReference

In [6]:
hn_dset = client.get_dataset(hn_dataset_ref)

In [7]:
type(hn_dset)

google.cloud.bigquery.dataset.Dataset

In [8]:
[x.table_id for x in client.list_tables(hn_dset)]

['comments', 'full', 'full_201510', 'stories']

In [9]:
hn_full = client.get_table(hn_dset.table('full'))

In [10]:
type(hn_full)

google.cloud.bigquery.table.Table

In [11]:
[command for command in dir(hn_full) if not command.startswith('_')]

['clustering_fields',
 'created',
 'dataset_id',
 'description',
 'encryption_configuration',
 'etag',
 'expires',
 'external_data_configuration',
 'friendly_name',
 'from_api_repr',
 'from_string',
 'full_table_id',
 'labels',
 'location',
 'modified',
 'num_bytes',
 'num_rows',
 'partition_expiration',
 'partitioning_type',
 'path',
 'project',
 'range_partitioning',
 'reference',
 'require_partition_filter',
 'schema',
 'self_link',
 'streaming_buffer',
 'table_id',
 'table_type',
 'time_partitioning',
 'to_api_repr',
 'to_bqstorage',
 'view_query',
 'view_use_legacy_sql']

In [12]:
hn_full.schema

[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', ()),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', ()),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ()),
 SchemaField(

In [13]:
schema_subset = [col for col in hn_full.schema if col.name in ('by', 'title', 'time')]
results = [x for x in client.list_rows(hn_full, start_index=100, selected_fields=schema_subset, max_results=10)]

In [14]:
schema_subset = [col for col in hn_full.schema if col.name in ('by', 'title', 'time')]
results = [x for x in client.list_rows(hn_full, start_index=100, selected_fields=schema_subset, max_results=10)]

In [15]:
for i in results:
    print(dict(i))

{'title': None, 'by': 'drewcrawford', 'time': 1438467238}
{'title': None, 'by': 'pibefision', 'time': 1262720406}
{'title': None, 'by': 'Recoil42', 'time': 1363216473}
{'title': None, 'by': 'sargun', 'time': 1364502918}
{'title': None, 'by': 'ams6110', 'time': 1536201953}
{'title': None, 'by': 'danielvinson', 'time': 1536966263}
{'title': None, 'by': 'rweichler', 'time': 1496257980}
{'title': None, 'by': 'feross', 'time': 1543718244}
{'title': None, 'by': 'Cthulhu_', 'time': 1417782911}
{'title': None, 'by': 'talkingquickly', 'time': 1428410268}


In [16]:
import pandas as pd

In [17]:
# pip install google-cloud-bigquery-storage

In [18]:
import google.cloud.bigquery.magics

google.cloud.bigquery.magics.context.use_bqstorage_api = True

In [19]:
from google.cloud import bigquery_storage_v1beta1


bqstorageclient = bigquery_storage_v1beta1.BigQueryStorageClient()

In [23]:
hn_comments = client.get_table(hn_dset.table('comments'))
hn_comments.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', 'Unique comment ID', ()),
 SchemaField('by', 'STRING', 'NULLABLE', 'Username of commenter', ()),
 SchemaField('author', 'STRING', 'NULLABLE', 'Username of author', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('time_ts', 'TIMESTAMP', 'NULLABLE', 'Human readable time in UTC (format: YYYY-MM-DD hh:mm:ss)', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Comment text', ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('ranking', 'INTEGER', 'NULLABLE', 'Comment ranking', ())]

In [20]:
# pip install pyarrow

In [29]:
# Download query results.



query_string = "SELECT * FROM `bigquery-public-data.hacker_news.comments`"
dataframe = (
    client.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)
print(dataframe.head())

         id   by author        time                   time_ts  \
0   2080961  rdl    rdl  1294431264 2011-01-07 20:14:24+00:00   
1   3721745  rdl    rdl  1332119567 2012-03-19 01:12:47+00:00   
2   5127745  rdl    rdl  1359363674 2013-01-28 09:01:14+00:00   
3  10105050  rdl    rdl  1440319572 2015-08-23 08:46:12+00:00   
4     46875  rms    rms  1188186242 2007-08-27 03:44:02+00:00   

                                                text    parent deleted  dead  \
0  I think there is some assumption that Amazon r...   2078093    None  None   
1  It's taken as a sign that the company doesn't ...   3721320    None  None   
2  The other question is -- do you want to be Top...   5127269    None  None   
3  Yeah, the pmarca&#x2F;thiel one was dramatical...  10105028    None  None   
4  Doesn't work in Firefox 3; it's still pretty c...     46872    None  None   

   ranking  
0        1  
1        1  
2        1  
3        1  
4        1  


In [46]:
dataframe.head()

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2080961,rdl,rdl,1294431264,2011-01-07 20:14:24+00:00,I think there is some assumption that Amazon r...,2078093,None,None,1
1,3721745,rdl,rdl,1332119567,2012-03-19 01:12:47+00:00,It's taken as a sign that the company doesn't ...,3721320,None,None,1
2,5127745,rdl,rdl,1359363674,2013-01-28 09:01:14+00:00,The other question is -- do you want to be Top...,5127269,None,None,1
3,10105050,rdl,rdl,1440319572,2015-08-23 08:46:12+00:00,"Yeah, the pmarca&#x2F;thiel one was dramatical...",10105028,None,None,1
4,46875,rms,rms,1188186242,2007-08-27 03:44:02+00:00,Doesn't work in Firefox 3; it's still pretty c...,46872,None,None,1


In [33]:
df = dataframe[['id', 'by', 'text']]

In [34]:
df.head()

,id,by,text
0,2080961,rdl,I think there is some assumption that Amazon r...
1,3721745,rdl,It's taken as a sign that the company doesn't ...
2,5127745,rdl,The other question is -- do you want to be Top...
3,10105050,rdl,"Yeah, the pmarca&#x2F;thiel one was dramatical..."
4,46875,rms,Doesn't work in Firefox 3; it's still pretty c...


In [42]:
df.shape

(8399417, 3)

In [35]:
small = df.sample(100)

In [36]:
small.shape

(100, 3)

In [38]:
small.to_json('small.json')

In [40]:
read = pd.read_json('small.json')

In [41]:
read.head()

,id,by,text
1053288,8817348,BrainInAJar,it is possible to agree with the general tone ...
1183776,1909793,MisterWebz,Try Reddit. Wider audience.
1193520,1088996,ThinkWriteMute,I'd love to see an educational micro-kernel OS...
1198632,368085,tlrobinson,Perhaps the developer already knows JavaScript...
1205539,4224529,smacktoward,If someone presents himself to me as a profess...


In [43]:
df.to_json('comments.json')

In [44]:
big_read = pd.read_json('comments.json')

In [45]:
big_read.head()

,id,by,text
0,2080961,rdl,I think there is some assumption that Amazon r...
1,3721745,rdl,It's taken as a sign that the company doesn't ...
10,2787312,sjs,It's not the best metaphor but it is a relativ...
100,9396033,nova,&gt; equivalent of low carb diets<p>But they a...
1000,1731262,metamemetics,I never affirmed or attacked or recognized you...


In [57]:
big_reset = big_read.reset_index(drop=True)

In [59]:
big_reset[big_reset['by']=='rdl'][:5]

,id,by,text
0,2080961,rdl,I think there is some assumption that Amazon r...
1,3721745,rdl,It's taken as a sign that the company doesn't ...
957,1758785,rdl,"Weird, I've never been charged cover at Old Pro."
958,5914252,rdl,UDP will give you better performance. TCP wil...
959,5114422,rdl,"Wow, I had no idea they were processing $10b/y..."
